In [24]:
from cryptocmd import CmcScraper
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
scraper = CmcScraper("DOGE")

In [3]:
headers, data = scraper.get_data()

In [4]:
xrp_json_data = scraper.get_data("json")

In [5]:
scraper.export("csv", name="doge_all_time")

In [6]:
df = scraper.get_dataframe()

In [7]:
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

In [8]:
df.head()

,Date,Open,High,Low,Close,Volume,Market Cap
2021-05-31,2021-05-31,0.301949,0.329163,0.294330,0.325810,2.440829e+09,4.229895e+10
2021-05-30,2021-05-30,0.302969,0.310324,0.287680,0.302224,1.929948e+09,3.923259e+10
2021-05-29,2021-05-29,0.311398,0.317918,0.281793,0.303110,2.470711e+09,3.934490e+10
2021-05-28,2021-05-28,0.333651,0.347879,0.301518,0.311205,3.289456e+09,4.039136e+10
2021-05-27,2021-05-27,0.353463,0.353463,0.328144,0.332945,2.485942e+09,4.320847e+10


In [9]:
df = df[['Close']]
df.tail()

,Close
2013-12-19,0.001162
2013-12-18,0.000362
2013-12-17,0.000269
2013-12-16,0.000205
2013-12-15,0.000298


In [10]:
prediction_days = 1
df['Prediction'] = df[['Close']].shift(-prediction_days)
df.head()

,Close,Prediction
2021-05-31,0.325810,0.302224
2021-05-30,0.302224,0.303110
2021-05-29,0.303110,0.311205
2021-05-28,0.311205,0.332945
2021-05-27,0.332945,0.353693


In [11]:
X = np.array(df.drop(['Prediction'], 1))
X = X[:len(df) - prediction_days - 1]
print(X)

[[3.25810450e-01]
 [3.02223590e-01]
 [3.03109710e-01]
 ...
 [1.16248999e-03]
 [3.61631013e-04]
 [2.68539996e-04]]


In [12]:
y = np.array(df['Prediction'])
y = y[:-prediction_days - 1]
print(y)

[3.02223590e-01 3.03109710e-01 3.11204870e-01 ... 3.61631013e-04
 2.68539996e-04 2.04730997e-04]


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [15]:
from sklearn.ensemble import RandomForestRegressor

In [16]:
forest = RandomForestRegressor(n_estimators=2,random_state=587)
forest.fit(X_train,y_train)

RandomForestRegressor(n_estimators=2, random_state=587)

In [17]:
print(forest.score(X_test,y_test))

0.8952162813464061


In [26]:
# Prediction on test data
prediction = forest.predict(X_test)
print(prediction)
print()
print(y_test)


[6.34775992e-04 2.56858969e-03 3.57450009e-03 1.28841002e-04
 3.28521647e-03 2.29469454e-03 1.43878002e-04 2.81621739e-03
 2.67587995e-04 2.25323690e-04 2.07896730e-03 2.03806974e-03
 1.60244002e-03 5.13691650e-02 1.51636494e-04 1.97149299e-03
 3.76360986e-04 2.20114510e-04 3.52824842e-03 1.33495028e-04
 2.28019999e-04 2.27257710e-04 6.62902999e-03 2.20114510e-04
 4.74983281e-03 2.19222500e-04 2.71868007e-03 2.73762139e-03
 3.23814101e-03 2.39722989e-03 3.89169300e-01 2.63559803e-03
 8.52792989e-04 5.00047000e-03 2.70210992e-04 2.14657666e-04
 3.07046008e-04 2.74601000e-03 1.99057709e-03 3.16386022e-03
 1.10858999e-03 6.01826981e-03 1.93436063e-03 5.00047000e-03
 2.66292183e-03 2.23592282e-04 1.42564502e-04 1.46288497e-04
 5.58689010e-03 1.98612004e-04 5.61913450e-02 1.66086947e-03
 1.31123998e-04 2.54276002e-03 2.59282009e-04 2.24076997e-04
 3.57154501e-03 3.59485000e-03 3.09277461e-03 8.40172980e-03
 2.54331011e-04 4.96956008e-03 2.44647992e-04 4.46296006e-04
 2.48570592e-03 2.606078

In [27]:
temp = df[:-prediction_days]
x_val = temp.tail(1)['Close'][0]
print(x_val)

0.00020473099721129984


In [29]:
prediction = forest.predict([[x_val]])
print('Price of Dogecoin in', prediction_days, 'day(s) is predicted to be', prediction)
print('The actual price was', temp.tail(1)['Prediction'][0])

Price of Dogecoin in 1 day(s) is predicted to be [0.00020955]
The actual price was 0.00029788099345751107
